<a href="https://colab.research.google.com/github/Vanbk/Tensorflow/blob/master/rnn_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function

import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from t

In [0]:
tf.reset_default_graph()

In [0]:
'''
To classify images using a recurrent neural network, 
we consider every image row as a sequence of pixels. 
Because MNIST image shape is 28*28px, we will then handle 
28 sequences of 28 timesteps for every sample.
'''

# Training Parameter
learning_rate = 0.001
training_steps = 10000
batch_size = 128
display_step = 200

# Network Parameters
num_input = 28 #MNIST image size = (28 * 28)
timesteps = 28 
#If the input 𝑥 is of size 𝑛×1, and we have 𝑑 hidden states, 
#then the size of the total weight matrix is 4𝑑×(𝑛+𝑑)
#(the total weight matrix includes parameters for all 4 
# operations and the sub matrices W and U)
num_hidden = 128 # hidden states
num_classes = 10 # MNIST total classes (0 - 9)

# tf Graph

X = tf.placeholder(tf.float32, [None,timesteps, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])

In [0]:
# Define weights for the last layer, which is used for classification
# RNN output node weights and biases

weights = {
    #Hidden layer weights => 2*n_hidden because of forward + backward cells
    'out': tf.Variable(tf.random_normal([num_hidden, num_classes]))
}

biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}


In [0]:
def RNN(x, weights, biases):
  # input shape x: (batch_size, timesteps, n_input)
  # Required shape: 'timesteps' tensor list of shape (batch_size, n_input)
  x = tf.unstack(x, timesteps, 1)

  # Define a lstm cell
  lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

  # Get lstm cell ouput
  outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

  # the last layer is a dense layer
  return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [0]:
# softmax classification
logits = RNN(X, weights, biases)
prediction = tf.nn.softmax(logits)

# reduce_mean <=> average
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
# 1 in argmax means horizontal
correct_pre = tf.equal(tf.arg_max(prediction, 1), tf.arg_max(Y, 1))
# tf cast convert correct_pre to float before dividing -> get a float accuracy
accuracy = tf.reduce_mean(tf.cast(correct_pre, tf.float32))

# Initialize the variables 
init = tf.global_variables_initializer()

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [0]:
sess = tf.Session()

sess.run(init)

for step in range (1, training_steps + 1):
  batch_x, batch_y = mnist.train.next_batch(batch_size)
  #reshape data to 28 * 28
  batch_x = batch_x.reshape((batch_size, timesteps, num_input))
  # Run iptimization op
  sess.run(optimizer, feed_dict={X:batch_x, Y: batch_y})

  if step % display_step == 0 or step == 1:
    loss, acc = sess.run([loss_op, accuracy ], feed_dict={X:batch_x, Y: batch_y})
    print ("Step: {}  batch_loss: {}  train_acc:{}".format(step, loss, acc))
print ("Training finish !")

# Calculate accuracy for 128 mnist test images
test_len = 128
test_data = mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
test_label = mnist.test.labels[:test_len]
test_acc =  sess.run(accuracy, feed_dict={X: test_data, Y: test_label})

Step: 1  batch_loss: 3.1400885581970215  train_acc:0.046875
Step: 200  batch_loss: 2.1707303524017334  train_acc:0.2109375
Step: 400  batch_loss: 2.0485823154449463  train_acc:0.2578125
Step: 600  batch_loss: 1.7530176639556885  train_acc:0.5
Step: 800  batch_loss: 1.675374150276184  train_acc:0.515625
Step: 1000  batch_loss: 1.6245977878570557  train_acc:0.46875
Step: 1200  batch_loss: 1.5314277410507202  train_acc:0.5
Step: 1400  batch_loss: 1.28299081325531  train_acc:0.6171875
Step: 1600  batch_loss: 1.436321496963501  train_acc:0.515625
Step: 1800  batch_loss: 1.3278319835662842  train_acc:0.5546875
Step: 2000  batch_loss: 1.3676435947418213  train_acc:0.6015625
Step: 2200  batch_loss: 1.222402811050415  train_acc:0.59375
Step: 2400  batch_loss: 1.2574543952941895  train_acc:0.5390625
Step: 2600  batch_loss: 1.195747971534729  train_acc:0.6484375
Step: 2800  batch_loss: 1.1344588994979858  train_acc:0.640625
Step: 3000  batch_loss: 1.0597310066223145  train_acc:0.65625
Step: 3200 

In [0]:
print ("Testing accuracy: ", test_acc)

Testing accuracy:  0.8828125
